# Assignment Part 1 Brief 

## Deadline: Friday, December 3, 2021 at 15:00 hrs

## Number of marks available: 25

## Scope: Sessions 1 to 5

### Please READ the whole assignment first, before starting to work on it.

### How and what to submit

A. A **Jupyter Notebook** with the code in all the cells executed.

B. Name your Notebook **COM4509-6509_Assignment_Part1_Username_XXXXXX.ipynb** where XXXXXX is your username such as such as abc18de.  

C. Upload a .zip file to Blackboard before the deadline that contains the Jupyter Notebook in B and any other files requested for the solution of Part 2. 

D. **NO DATA UPLOAD**: Please do not upload the data files used in this Notebook. We have a copy already. 


### Assessment Criteria 

* Being able to use numpy and pandas to preprocess a dataset.

* Being able to use numpy to build a machine learning pipeline for supervised learning. 

* Being able to follow the steps involved in an end-to-end project in machine learning.

* Being able to use scikit-learn to design a machine learning model pipeline


### Late submissions

We follow Department's guidelines about late submissions, i.e., a deduction of 5% of the mark each working day the work is late after the deadline. NO late submission will be marked one week after the deadline because we will release a solution by then. Please read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/late-submission) if you are taking COM4509 or read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/assessment/late-submission) if you are taking COM6509. 

### Use of unfair means 

**"Any form of unfair means is treated as a serious academic offence and action may be taken under the Discipline Regulations."** (from the students Handbook). Please carefully read [this link](https://sites.google.com/sheffield.ac.uk/comughandbook/general-information/assessment/unfair-means) on what constitutes Unfair Means if not sure, for COM4509. If you are taking COM6509, please read [this link](https://sites.google.com/sheffield.ac.uk/compgtstudenthandbook/menu/referencing-unfair-means) if you are not sure what is Unfair means. If you still have questions, please ask your Personal tutor or the Lecturers. 

# A dataset of air quality

The dataset you will use in this assignment comes from a popular machine learning repository that hosts open source datasets for educational and research purposes, the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php). We are going to use regularised ridge regression and random forests for predicting air quality. The description of the dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

In [1]:
import urllib.request
doq = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
pat_sav = "./AirQualityUCI.zip"
urllib.request.urlretrieve(doq, pat_sav)
#urllib.request.urlretrieve("https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip")

('./AirQualityUCI.zip', <http.client.HTTPMessage at 0x7fddfe1eb6a0>)

In [2]:
import zipfile
zip = zipfile.ZipFile('./AirQualityUCI.zip', 'r')
for name in zip.namelist():
    zip.extract(name, '.')

In [3]:
# The .csv version of the file has some typing issues, so we use the excel version
import pandas as pd 
air_quality_full = pd.read_excel('./AirQualityUCI.xlsx', usecols=range(2,15))

We can see some of the rows in the dataset 

In [4]:
air_quality_full.sample(5)

,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
4618,0.9,829.75,-200,3.786142,698.75,90.0,1011.75,54.0,1338.75,736.00,18.775,59.474999,1.275761
1932,0.1,780.75,-200,2.888862,645.00,32.0,1295.00,45.0,1319.75,575.25,17.275,47.550000,0.929469
4056,1.9,-200.00,-200,-200.000000,-200.00,103.0,-200.00,88.0,-200.00,-200.00,-200.000,-200.000000,-200.000000
8105,4.7,-200.00,-200,-200.000000,-200.00,746.1,-200.00,275.9,-200.00,-200.00,-200.000,-200.000000,-200.000000
6128,0.5,888.75,-200,4.932008,760.00,195.0,967.75,120.0,918.75,983.25,5.275,48.674999,0.436444


The target variable corresponds to the CO(GT) variable of the first column. The following columns correspond to the variables in the feature vectors, *e.g.*, PT08.S1(CO) is $x_1$ up until AH which is $x_D$. The original dataset also has a date and a time columns that we are not going to use in this assignment.

### Removing instances 

The dataset has missing values tagged with a -200 value. To simplify the design of the machine learning models in this assignment, we perform the following two operations to the dataset right from the beginning:

* we will remove the rows for which the target variable has missing values. We are doing supervised learning so we need all our data observations to have known target values.

* we will remove features with more than 20% of missing values. 

The code below performs both operations

In [5]:
# We first remove the rows for which there are missing values in the target feature
air_quality = air_quality_full.loc[air_quality_full.iloc[:, 0]!=-200, :]

In [6]:
# We now remove the columns (features) for which there are more that 20% of missing values
import numpy as np
ndata, ncols = np.shape(air_quality) # number of data observations and number of columns in the dataframe
pmissing = np.empty(ncols)         # An empty vector that will keep the percentage of missing values per feature
for i in range(ncols):
    pmissing[i] = (air_quality.iloc[:, i]==-200).sum()/ndata # Computes the percentage of missing values per column
air_quality = air_quality.loc[:, pmissing < 0.2]

### Splitting the dataset 

Before designing any machine learning model, we need to set aside the test data. We will use the remaining training data for fitting the model. *It is important to remember that the test data has to be set aside before preprocessing*. 

Any preprocessing that you do has to be done only on the training data and several key statistics need to be saved for the test stage.  Separating the dataset into training and test before any preprocessing has happened help us to recreate the real world scenario where we will deploy our system and for which the data will come without any preprocessing.

Furthermore, we are going to use *hold-out validation* for validating our predictive model so we need to further separate the training data into a training set and a validation set.

We split the dataset into a training set, a validation set and a test set. The training set will have 70% of the total observations, the validation set 15% and the test set, the remaining 15%. For making the random selections of the training and validation sets **make sure that you use a random seed that corresponds to the last five digits of your student UCard**. In the code below, I have used 55555 as an example of my random seed.

In [7]:
np.random.seed(55555)                 # Make sure you use the last five digits of your student UCard as your seed
index = np.random.permutation(ndata)  # We permute the indexes 
N = np.int64(np.round(0.70*ndata))    # We compute N, the number of training instances
Nval = np.int64(np.round(0.15*ndata)) # We compute Nval, the number of validation instances   
Ntest = ndata - N - Nval              # We compute Ntest, the number of test instances
data_training_unproc = air_quality.iloc[index[0:N], :].copy() # Select the training data
data_val_unproc = air_quality.iloc[index[N:N+Nval], :].copy() # Select the validation data
data_test_unproc = air_quality.iloc[index[N+Nval:ndata], :].copy() # Select the test data

The assigment is divided into two sections. In the **first section**, you will design a regularised ridge regression model trained with stochastic gradient descent. You will write all the code from scratch. You should not use any library that already implements any of the routines considered in this section, for example, scikit-learn. In the **second section**, you will desing a random forests model and you are allowed to use scikit-learn in this section. 

# 1. Using regularised ridge regression to predict air quality (12 marks)

**DO NOT USE scikit-learn for the questions on this section. You are meant to write Python code from scratch. Using scikit-learn for the questions in this section will give ZERO marks. No excuse will be accepted.**

When writing your code, you will find out that there are operations that are repeated at least twice. We will assign marks for use of Python functions and for commenting your code. The marks will be assigned as:

* Did you include Python functions to solve the question and avoid repeating code? (**1 mark**)
* Did you comment your code to make it readable to others? (**1 mark**)

Regularisation is a technique commonly used in Machine Learning to prevent overfitting. It consists on adding terms to the objective function such that the optimisation procedure avoids solutions that just learn the training data. Popular techniques for regularisation in Supervised Learning include [Lasso Regression](https://en.wikipedia.org/wiki/Lasso_(statistics)), [Ridge Regression](https://en.wikipedia.org/wiki/Tikhonov_regularization) and the [Elastic Net](https://en.wikipedia.org/wiki/Elastic_net_regularization). 

In this part of the Assignment, you will be looking at Ridge Regression and implementing equations to optimise the objective function using the update rules for stochastic gradient descent. You will use those update rules for making predictions on the Air Quality dataset.

## 1.1 Ridge Regression

Let us start with a data set for training $\mathcal{D} = \{\mathbf{y}, \mathbf{X}\}$, where the vector $\mathbf{y}=[y_1, \cdots, y_N]^{\top}$ and $\mathbf{X}$ is the design matrix from Lab 4, this is, 

\begin{align*}
    \mathbf{X} = 
                \begin{bmatrix}
                        1 & x_{1,1} & \cdots & x_{1, D}\\
                        1 & x_{2,1} & \cdots & x_{2, D}\\
                   \vdots &  \vdots\\
                        1 & x_{N,1} & \cdots & x_{N, D}
                \end{bmatrix}
               = 
               \begin{bmatrix}
                      \mathbf{x}_1^{\top}\\
                       \mathbf{x}_2^{\top}\\
                          \vdots\\
                        \mathbf{x}_N^{\top}
                \end{bmatrix}.
\end{align*}

Our predictive model is going to be a linear model

$$ f(\mathbf{x}_i) = \mathbf{w}^{\top}\mathbf{x}_i,$$

where $\mathbf{w} = [w_0\; w_1\; \cdots \; w_D]^{\top}$.

The **objective function** we are going to use has the following form

$$ E(\mathbf{w}, \lambda) = \frac{1}{N}\sum_{n=1}^N (y_n - f(\mathbf{x}_n))^2 + \frac{\lambda}{2}\sum_{j=0}^D w_j^2,$$

where $\lambda>0$ is known as the *regularisation* parameter.

This objective function was studied in Lecture 4. 

The first term on the rhs is what we call the "fitting" term whereas the second term in the expression is the regularisation term. Given $\lambda$, the two terms in the expression have different purposes. The first term is looking for a value of $\mathbf{w}$ that leads the squared-errors to zero. While doing this, $\mathbf{w}$ can take any value and lead to a solution that it is only good for the training data but perhaps not for the test data. The second term is regularising the behavior of the first term by driving the $\mathbf{w}$ towards zero. By doing this, it restricts the possible set of values that $\mathbf{w}$ might take according to the first term. The value that we use for $\lambda$ will allow a compromise between a value of $\mathbf{w}$ that exactly fits the data (first term) or a value of $\mathbf{w}$ that does not grow too much (second term).

This type of regularisation has different names: ridge regression, Tikhonov regularisation or $\ell_2$ norm regularisation. 

## 1.2 Optimising the objective function with respect to $\mathbf{w}$

There are two ways we can optimise the objective function with respect to $\mathbf{w}$. The first one leads to a closed form expression for $\mathbf{w}$ and the second one using an iterative optimisation procedure that updates the value of $\mathbf{w}$ at each iteration by using the gradient of the objective function with respect to $\mathbf{w}$,
$$
\mathbf{w}_{\text{new}} = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}},
$$
where $\eta$ is the *learning rate* parameter and $\frac{d E(\mathbf{w}, \lambda)}{d\mathbf{w}}$ is the gradient of the objective function.

It can be shown (this is a question in the Exercise Sheet 4) that a closed-form expression for the optimal $\mathbf{w}_*$ is given as

\begin{align*}            
            \mathbf{w}_*& = \left(\mathbf{X}^{\top}\mathbf{X} + \frac{\lambda N}   
                                     {2}\mathbf{I}\right)^{-1}\mathbf{X}^{\top}\mathbf{y}.
\end{align*}

Alternatively, we can find an update equation for $\mathbf{w}_{\text{new}}$ using gradient descent leading to:

\begin{align*}
   \mathbf{w}_{\text{new}} & = \mathbf{w}_{\text{old}} - \eta \frac{d E(\mathbf{w}, \lambda)}
                              {d\mathbf{w}},\\
                           & = \mathbf{w}_{\text{old}} +  \frac{2\eta}{N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n  
                       - \eta\lambda\mathbf{w}_{\text{old}}\\
                           & = (1 - \eta\lambda)\mathbf{w}_{\text{old}} + \frac{2\eta}
                               {N}\sum_{n=1}^N   
                               \left(y_n - \mathbf{x}_n^{\top}\mathbf{w}_{\text{old}}\right)\mathbf{x}_n
\end{align*}

## 1.3 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 1.a: Missing values and standardisation (2 marks)

* For all the other features with missing values, use the mean value of the non-missing values for that feature to perform imputation. Save these mean values, you will need them when performing the validation stage (**1 mark**).

* Once you have imputed the missing data, we need to standardise the input vectors. Standardise the training data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Keep the mean values and standard deviations, you will need them at validation time (**1 mark**).

#### Question 1.a Answer

In [8]:
# Write your code here
# Get the mean value of the non-missing values per column and use it to impute that column
def get_stats_missing_data(data_local):
    """
    Takes an input a dataset with missing values, and replace the missing values by the mean value 
    of the non-missing. Returns the new dataset with imputed values and the missing values used for imputation
    """
    
    data_local = data_local.copy()
    ncols = np.size(data_local, 1) # We need to count again the number of columns after removing those with >20% missing values
    no_missing_means = np.empty(ncols) # We save the mean values used for imputation in this vector.
    #names_columns = data_local.columns
    for i in range(0, ncols):    
        # We're getting the mean values of the non-missing values
        
        #value_mean = np.mean(data_local.loc[data_local.iloc[:, i]!=-200, names_columns[i]]) 
        
        value_mean = np.mean(data_local.iloc[(data_local.iloc[:, i]!=-200).values,i])
        
        # We do the imputation
        data_local.iloc[(data_local.iloc[:, i]==-200).values, i] = value_mean 
        #print(data_local.iloc[0, :])

        # We save the mean values for later
        no_missing_means[i] = value_mean
    return data_local, no_missing_means

# Standardise the data
def get_stats_standardisation(data_local):
    """
    Takes an input dataframe and computes the mean and standard deviation per column and use those values 
    to standardise each column. It assumes the target feature is the first column.
    """
    uX =  (data_local.iloc[:, 1:ncols]).values # We get the values of the features
    N2, D = np.shape(uX)
    meansX = np.mean(uX, 0) # We get the mean values for each column 
    stdX = np.std(uX, 0)    # We compute the standard deviation for each column
    nX = np.empty((N2, D))
    for i in range(D):      # We go around substracting the mean and dividing by the std for each feature 
        nX[:, i] = (uX[:, i] - meansX[i])/stdX[i]  
    return nX, meansX, stdX

In [9]:
data_training, no_missing_means = get_stats_missing_data(data_training_unproc)
nX, meansX, stdX = get_stats_standardisation(data_training)

## 1.4 Training and validation stages

We have now curated our training data by removing data observations and features with a large amount of missing values. We have also normalised the feature vectors. We are now in a good position to work on developing the prediction model and validating it. We will use gradient descent for iterative optimisation. 

We first organise the dataframe into the vector of targets $\mathbf{y}$, call it `yTrain`, and the design matrix $\mathbf{X}$, call it `XTrain`.

In [10]:
yTrain = np.reshape((data_training.iloc[:, 0]).values, (N,1)) # The traininig target labels
XTrain = np.concatenate((np.ones((N,1)), nX), axis=1)         # The standardised inputs with an additional column vector  

### Question 1.b: finding the optimal $\mathbf{w}$ with stochastic gradient descent (3 marks)

Use gradient descent to iteratively compute the value of $\mathbf{w}_{\text{new}}$. Instead of using all the training set in `XTrain` and `yTrain` to compute the gradient, use a subset of $S$ instances in `XTrain` and `yTrain`. This is sometimes called *minibatch gradient descent* where $S$ is the size of the minibacth. When using gradient descent with minibatches, you need to find the best values for three parameters: $\eta$, the learning rate, $S$, the number of datapoints in the minibatch and $\gamma$, the regularisation parameter.

* In this question we will use the validation data. So before proceeding to the next steps, make sure that you:  replace the missing values on each feature variables with the mean value you computed with the training data; standardise the validation data using the means and standard deviations computed from the training data (**1 mark**)
    
* Create a grid of values for the parameters $\gamma$ and $\eta$ using `np.logspace` and a grid of values for $S$ using `np.linspace`. Because you need to find three parameters, start with five values for each parameter in the grid and see if you can increase it. Make sure you understand what is the meaning of `np.logspace` and `np.linspace`. Notice that you can use negative values for `start` in `np.logspace` (**1 mark**)

* For each value that you have of $\gamma$, $\eta$ and $S$ from the previous step, use the training set to compute $\mathbf{w}$ using minibatch gradient descent and then measure the RMSE over the validation data. For the validation data, make sure you preprocess it before applying the prediction model over it. For the minibatch gradient descent choose to stop the iterative procedure after $200$ iterations.

* Choose the values of $\gamma$, $\eta$ and $S$ that lead to the lowest RMSE and save them. You will use them at the test stage.

#### Question 1.b Answer

In [11]:
# Write your code here
# PREPROCESSING THE VALIDATION DATA
def set_missing_data(data_local, no_missing_means):
    data_local = data_local.copy()
    ncols = np.size(data_local, 1)
    for i in range(1, ncols):            
        data_local.iloc[(data_local.iloc[:, i]==-200).values, i] = no_missing_means[i]
    return data_local

def set_standardisation(data_local, meansX, stdX):
    uX =  (data_local.iloc[:, 1:ncols]).values
    N2, D = np.shape(uX)    
    nX = np.empty((N2, D))
    for i in range(D):
        nX[:, i] = (uX[:, i] - meansX[i])/stdX[i]
    return nX    

# We replace the missing values on each feature variable with the mean of the features before standardisation calling
# the function set_missing_data
data_val = set_missing_data(data_val_unproc, no_missing_means)        
# We now standardise the validation data by calling the function set_standardisation
nXVal = set_standardisation(data_val, meansX, stdX)
# We set the targets and build the design matrix for the test data
yVal = np.reshape((data_val.iloc[:, 0]).values, (Nval,1))
XVal = np.concatenate((np.ones((Nval,1)), nXVal), axis=1)

In [12]:
# CREATE THE GRID OF VALUES FOR GAMMA, ETA AND S
num = 5
gamma_vector = np.logspace(-3, -1, num)
eta_vector = np.logspace(-3, -1, num)
S_vector = np.linspace(50, 1000, num)

In [13]:
# TRAIN THE REGULARISED LINEAR MODEL AND COMPUTE THE RMSE FOR ALL VALUES OF GAMMA, ETA AND S

# D: number of features in the data + 1 (for the column of 1s in the training dataset)

def sgd_optimiser(X, y, gamma, eta, S, N, D, max_iters):
    w = np.random.normal(loc=0.0, scale=1.0, size= [D, 1])
    for iters in range(max_iters):
    # We first choose a random subset S of the available training set
        index = np.random.permutation(N)
        Xs = X[index[0:S], :]
        ys = y[index[0:S], :]
        gradient = -(2/S)*(Xs.T@ys) + (2/S)*(Xs.T@Xs@w) + gamma*w
        w = w - eta*gradient
    return w

max_iters = 100
D = np.size(XTrain, 1)
rmse_val = np.empty([num, num, num])
for i in range(num):
    for j in range(num):
        for k in range(num):
            gamma = gamma_vector[i]
            eta = eta_vector[j]
            S = np.int64(np.floor(S_vector[k]))
            w = sgd_optimiser(XTrain, yTrain, gamma, eta, S, N, D, max_iters)            
            fVal = XVal@w
            eVal = yVal - fVal
            rmse_val[i, j, k] = np.sqrt(np.mean(eVal*eVal)) # Compute the MSE for all combinations 
best_params = np.argwhere(rmse_val == np.min(rmse_val))
gamma = gamma_vector[best_params[0][0]]
eta   = eta_vector[best_params[0][1]]
S = np.int64(np.floor(S_vector[best_params[0][2]]))

## 1.5 Test stage 

We now know which one is the best model, according to the validation data. We will now put together the training data and the validation data, perform the preprocessing as we did before, this is, treat the missing values and standardise the inputs. We will train the model again using the minibatch stochastic gradient descent and finally compute the RMSE over the test data.


### Question 1.c: combine the original training and original validation data and perform the preprocessing again to this new data (2 marks)

Put together the original training and validation dataset and perform the same preprocessing steps than before, these are: 

* for each feature, impute the missing values with the mean values of the non-missing values (**1 mark**) 

* stardardise the new training set (**1 mark**).

#### Question 1.c Answer

In [14]:
data_training_full = air_quality.iloc[index[0:N+Nval], :].copy() # Select the training data
# We go again through the features replacing the missing values with the average of the no missing values.
data_training_full, no_missing_means = get_stats_missing_data(data_training_full)
# Standardisation
nX2, meansX2, stdX2 = get_stats_standardisation(data_training_full)
# Create the new training set
Nfull = np.size(data_training_full, 0)
yTrain2 = np.reshape((data_training_full.iloc[:, 0]).values, (Nfull,1)) # The traininig target labels
XTrain2 = np.concatenate((np.ones((Nfull,1)), nX2), axis=1)         # The normalised inputs with an additional column vector  

### Question 1.d: Preprocess the test data, train the model and predict over the test data (3 marks)

Preprocess the test data and train a new model using the new training set. Finally, report the RMSE over the test set:

* Preprocess the test data by imputing the missing data and standardising it (**1 mark**). 

* Use the best values of $\gamma$, $\eta$ and $S$ found in the validation set and train a new regularised linear model with stochastic gradient descent (**1 mark**)

* Report the RMSE over the test data (**1 mark**)

#### Preprocessing over the test data

In [15]:
# Write your code here
# PREPROCESSING THE TEST DATA
# We replace the missing values on each feature variable with the mean of the features before standardisation
data_test = set_missing_data(data_test_unproc, no_missing_means)      
# We now standardise the test data
nXTest = set_standardisation(data_test, meansX2, stdX2)
# We set the targets and build the design matrix for the test data
yTest = np.reshape((data_test.iloc[:, 0]).values, (Ntest,1))
XTest = np.concatenate((np.ones((Ntest,1)), nXTest), axis=1)

#### We train a linear regularised regression model using the best parameters we found on the validation set and compute the RMSE over the test data

In [16]:
# TRAIN THE REGULARISED LINEAR MODEL AND COMPUTE THE RMSE FOR THE VALUE OF GAMMA, ETA AND S OVER THE TEST DATA
max_iters = 100
N2, D = np.shape(XTrain2)
w = sgd_optimiser(XTrain2, yTrain2, gamma, eta, S, N2, D, max_iters) 
fTest = XTest@w
eTest = yTest - fTest
rmse_test = np.sqrt(np.mean(eTest*eTest)) # Compute the RMSE  
print('The RMSE for ridge regression is:', rmse_test)

The RMSE for ridge regression is: 0.5349058939440109


# 2. Random forests (13 marks)

**USE scikit-learn for the questions on this section.**

In section 1, you used a regularised ridge regression model trained with SGD to create a linear predictive model. In this part of the assignment, you will use **scikit-learn** to train a random forest for regression over the air quality dataset.

## 2.1 Preprocessing the data

As mentioned before, the dataset has missing values tagged with a -200 value. Before doing any work with the training data, we want to make sure that we deal properly with the missing values. Furthermore, once we have dealt with the missing values, we want to standardise the training data. 

### Question 2.a: Pipeline for missing values and standardisation (3 marks)

* Employ the `SimpleImputer` method in Scikit-learn to impute the missing values in each column using the mean value of the non-missing values, instead (**1 mark**).

* Standardise the data by substracting the mean value for each feature and dividing the result by the standard deviation of each feature. Employ the `StandardScaler` method (**1 mark**).

* Create a `Pipeline` that you can use to preprocess the data by filling missing values and then standardising the features (**1 mark**).

#### Question 2.a Answer

In [18]:
# Write your code here
# WE FIRST IMPORT THE RELEVANT METHODS
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [19]:
# WE CAN NOW CREATE THE PIPELINE
preprocess = Pipeline([('imputer', SimpleImputer(missing_values=-200, strategy='mean')), 
                       ('scaler', StandardScaler())])

### Question 2.b: Use the Pipeline to fit the training data and transform the validation data (2 marks)

In the previous question, you created a `Pipeline` for applying a `SimpleImputer` and a `StandardScaler`. Use the Pipeline to fit the training data (**1 mark**) and transform the validation data (**1 mark**).

In [20]:
# Write your code here
# FIT THE TRANSFORMER TO THE TRAININIG DATA
XTrain = data_training_unproc.iloc[:,1:ncols]
XTrain_transformed = preprocess.fit_transform(XTrain)
XVal = data_val_unproc.iloc[:,1:ncols]
XVal_transformed = preprocess.transform(XVal)

## 2.2 Random forest to predict air quality 

We now use random forests to predict air quality. Remember that the tree ensemble in random forests is built by training individual regression trees on different subsets of the training data and using a subset of the available features. For regression, the prediction is the average of the individual predictions of each tree. Some of the parameters required in the Random Forest implementation in Scikit-learn include:

Some of the additional parameters required in the Random Forest implementation in Scikit-learn include

> **n_estimators** the total number of trees to train<p>
**max_features** number of features to use as candidates for splitting at each tree node. <p>
    **boostrap**: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.<p>
   **max_samples**: If bootstrap is True, the number of samples to draw from X to train each base estimator.

### Question 2.c: train a random forest (4 marks)

In this question, you will train a random forest for predicting over the validation data. Use cross-validation over the validation data to select the best set of paramaters for the random forest regressor. Parameters to include in your exploration are **n_estimators**, **max_features** and **max_samples**. Use `np.linspace` or `np.logspace` to define ranges of values to explore for each parameter and create a grid to be assessed over the validation data. Make sure you use the same validation data that was given to you.

* Use `PredefinedSplit` to tell the cross validator which instances to use for training and which ones for validation (**1 mark**).

* Create a grid of values to explore that include a range of at least five values for each parameter **n_estimators**, **max_features** and **max_samples** (**1 mark**). 

* Train a random forest for regression model that uses the grid of parameters you created before. Use `GridSearchCV` to find the best set of parameters by performing cross-validation over the predefined split. (**1 mark**).

* Print the best values in your grid for **n_estimators**, **max_features** and **max_samples** (**1 mark**).

In [21]:
# Write your code here
# PREDEFINED SPLIT
from sklearn.model_selection import PredefinedSplit
test_fold = np.zeros((N+Nval, 1))
test_fold[0:N] = -1
ps = PredefinedSplit(test_fold)
# Concatenate the train and validation inputs and outputs
XTrain_Val = np.vstack((XTrain, XVal))
yTrain = np.reshape((data_training.iloc[:, 0]).values, (N,1)) 
yVal   = np.reshape((data_val.iloc[:, 0]).values, (Nval,1)) 
yTrain_Val = np.vstack((yTrain, yVal))

In [22]:
# CREATE A GRID OF PARAMETER VALUES FOR n_estimators, max_features and max_samples
n_features = np.size(XTrain_Val, 1)
n_opts_feats = 5
n_opts_ests = 5
n_opts_samples = 5
grid_features = np.int64(np.floor(np.linspace(1, n_features, n_opts_feats)))
grid_estimators = np.int64(np.floor(np.linspace(10, 100, n_opts_ests)))
grid_samples = np.linspace(0.1, 0.9, n_opts_samples)
parameters = {'n_estimators': grid_estimators, 'max_features': grid_features, 'max_samples': grid_samples}

In [23]:
# TRAIN THE RANDOM FOREST FOR REGRESSION MODEL
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
grid_regression = GridSearchCV(RandomForestRegressor(), 
                               param_grid=parameters, cv=ps, scoring='neg_mean_squared_error')
grid_regression.fit(XTrain_Val, yTrain_Val.ravel())

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=RandomForestRegressor(),
             param_grid={'max_features': array([ 1,  3,  6,  8, 11], dtype=int64),
                         'max_samples': array([0.1, 0.3, 0.5, 0.7, 0.9]),
                         'n_estimators': array([ 10,  32,  55,  77, 100], dtype=int64)},
             scoring='neg_mean_squared_error')

In [24]:
# PRINT THE BEST PARAMETERS
best_n_estimators = grid_regression.best_params_['n_estimators']
best_max_features = grid_regression.best_params_['max_features']
best_max_samples = grid_regression.best_params_['max_samples']
print('Best n_estimator', best_n_estimators)
print('Best max features', best_max_features)
print('Best max samples', best_max_samples)

Best n_estimator 100
Best max features 6
Best max samples 0.9


### Question 2.d: train a new model over the whole training data and report the prediction over the test set (4 marks)


Now that we have identified the best paramaters of the regression model, we use these parameters to train a new model over the whole training data (`data_training` plus `data_val`). We apply this model to the test set and report the performance.

* Create a new preprocessing pipeline for taking care of the missing values and standardisation over the whole training data (**1 mark**)

* Apply the created preprocessing pipeline to the test data (**1 mark**)

* Fit a random forest regression model to the training data using the best parameters found in Question 2.c (**1 mark**)

* Compute the RMSE over the test data and report the result (**1 mark**)

In [25]:
# Write your code here
# PREPROCESSING PIPELINE OVER THE WHOLE TRAINING DATA
data_training_full = air_quality.iloc[index[0:N+Nval], :].copy() # Select the training data
XTrainFull = data_training_full.iloc[:,1:ncols]
XTrainFull_transformed = preprocess.fit_transform(XTrainFull)
yTrainFull = data_training_full.iloc[:,0]

In [26]:
# APPLY THE PIPELINE TO THE TEST DATA
XTest = data_test_unproc.iloc[:,1:ncols]
XTest_transformed = preprocess.transform(XTest)

In [27]:
# FIR THE RANDOM FOREST REGRESSION MODEL
rforest_reg = RandomForestRegressor(n_estimators= best_n_estimators, max_features= best_max_features, 
                                    max_samples= best_max_samples)
trained_model = rforest_reg.fit(XTrainFull_transformed, yTrainFull)

In [28]:
# PREDICT OVER THE TEST DATA
pred_rfr = trained_model.predict(XTest_transformed)
yTest2 = data_test.iloc[:,0]
eTest = yTest2 - pred_rfr
rmse_test = np.sqrt(np.mean(eTest*eTest)) # Compute the RMSE 
print('The RMSE for random forest regression is:', rmse_test)

The RMSE for random forest regression is: 0.40823485531749426
